In [1]:
pip install transformers datasets evaluate jiwer accelerate==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirem

In [17]:
from transformers import AutoModelForCTC
from transformers import AutoConfig
from transformers import AutoFeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import AutoTokenizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_checkpoint = "phatjk/wav2vec2-large-vi"

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None


feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)


tokenizer = AutoTokenizer.from_pretrained(
  model_checkpoint,
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
model = AutoModelForCTC.from_pretrained(
    model_checkpoint,
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
).to(device)


`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.


In [18]:
from datasets import load_dataset, load_metric, Audio
data = load_dataset("common_voice", "vi", split="train[:100]")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


In [19]:
data = data.cast_column("audio", Audio(sampling_rate=16_000))

In [20]:
# Chon audio de test
import IPython.display as ipd
import numpy as np
import random
audio_num = random.randint(0, 100)
ipd.Audio(data=data[audio_num]["audio"]["array"], autoplay=False, rate=16000)

In [21]:
torch.tensor(data[audio_num]["audio"]["array"])

tensor([ 4.0927e-12, -1.1369e-12, -1.1369e-12,  ...,  5.1510e-06,
         7.5734e-07, -2.5075e-06], dtype=torch.float64)

In [22]:
# Predict
from transformers import AutoModelForCTC
import torch

inputs = processor(torch.tensor(data[audio_num]["audio"]["array"]), sampling_rate=16000, return_tensors="pt")
inputs = inputs.to(device)
with torch.no_grad():
    logits = model(**inputs).logits
    # Ket qua du doan
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    print("Result: ",transcription)

Result:  ['vậy con có cần mang hòn đá này bên người nữa không hả mẹ']
